In [4]:
import xml.etree.ElementTree as ET
import pandas as pd

from google.cloud import bigquery
# from google.cloud import bigquery_storage

from time import sleep
import pandas as pd
# import table_schema as table_schema
import pytz
# import qfrm
from datetime import datetime,timedelta
# from send_email import send_error_email
# from google.api_core.exceptions import BadRequest
# from google.cloud import secretmanager
from tqdm import tqdm, tqdm_notebook
import os
tqdm_notebook().pandas()
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy.optimize as optimize
sns.set()
from dateutil.relativedelta import relativedelta
import math
from pandas import NaT
from lxml import etree
import pandas as pd
from io import StringIO, BytesIO
from google.cloud import bigquery
import numpy as np
import xml.etree.ElementTree as ET

<ipython-input-4-8083338b643a>:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  tqdm_notebook().pandas()


/Users/FICC.AI/opt/anaconda3/lib/python3.8/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [5]:
path = "../../gsm_init_muni_APFICC_GSMF10I.1.1_1.20201203T1300-05.xml"

In [ ]:
def get_call_schedule(author):
    for doc in author.iter('instrument'):
        # print(doc.attrib["id"])
        doc_dict = {}
        call_details_info = doc.find(".//debt/call_details")
        if call_details_info is not None:
            for element in call_details_info:
                if element.tag == "call_schedule":
#                 new_dict.update({element.tag:element.text})
                    new_dict = {}
                    new_dict.update({"instrument_id":doc.attrib["id"]})
                    new_dict.update({element.find(".//call_price").tag:element.find(".//call_price").text})
                    new_dict.update({element.find(".//call_date").tag:element.find(".//call_date").text})
                    if element.find(".//cav_call_price") is not None:
                        new_dict.update({element.find(".//cav_call_price").tag:element.find(".//cav_call_price").text})
                    yield new_dict

In [6]:
def get_sink_schedule(author):
    for doc in author.iter('instrument'):
        sink_schedule = doc.iterfind(".//debt/sink_schedule")
        if sink_schedule is not None:
            for element in sink_schedule:
                new_dict = {}
                new_dict.update({"instrument_id":doc.attrib["id"]})
                if element.find(".//mandatory_sink_amount") is not None:
                    new_dict.update({element.find(".//mandatory_sink_amount").tag:element.find(".//mandatory_sink_amount").text})
                if element.find(".//mandatory_sink_date") is not None:
                    new_dict.update({element.find(".//mandatory_sink_date").tag:element.find(".//mandatory_sink_date").text})
                if element.find(".//mandatory_sink_price") is not None:
                    new_dict.update({element.find(".//mandatory_sink_price").tag:element.find(".//mandatory_sink_price").text})


In [3]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../creds.json"

In [2]:
PROJECT = 'eng-reactor-287421'
dataset = 'FULL_ICE'
table = 'sink_schedule'

In [7]:
bqclient = bigquery.Client(project=PROJECT,)

In [8]:
def create_table_with_schema(bq,project_id,dataset,table_id,schema = []):
    PROJECT = project_id
    bq = bq
    table_id = '{}.{}.{}'.format(PROJECT,dataset,table_id)
    table = bq.create_table(table_id, exists_ok=True)
    print('{} created on {}'.format(table.table_id, table.created))
    table = bq.get_table(table_id)
    table.schema = schema
    table = bq.update_table(table, ["schema"])

def load_data(bq,data,project,dataset,table):
    bq = bq
    table_id = '{}.{}.{}'.format(project,dataset,table)
    job_config = bigquery.LoadJobConfig(schema =[])
    job = bq.load_table_from_dataframe(data, table_id,job_config=job_config)

    try:
        job.result() # Waits for the job to complete.
        return 'success'  
    except BadRequest as ex:
        print(ex) 

In [20]:
create_table_with_schema(bqclient,PROJECT,dataset,table,)

sink_schedule created on 2020-12-14 17:30:54.125000+00:00


In [10]:
def data_loaded_threading(bqclient,doc_df,PROJECT,dataset,table,count):
    try:
        test = load_data(bqclient,doc_df,PROJECT,dataset,table)
        print(test)
    except Exception as e:
        print(e)
        for columns in doc_df.columns:
            print(columns)
        print(doc_df.head)
        print(count)
    

In [21]:
def sink_schedule(new_elem,final_list):
    
    list_of_elem = new_elem.xpath("//instrument/debt")
    instrument_id = new_elem.xpath("//instrument")[0].attrib["id"]
    if len(list_of_elem)>0:
        for element in list_of_elem[0].getchildren():
            if element.tag == "sink_schedule":
                new_dict = {}
                new_dict.update({"instrument_id":instrument_id})
#                 print("inside call schedule")
#                 print(len(element.getchildren()))
#                 new_dict.update({element.tag:element.text})
                for childs in element.getchildren():
# 
                    new_dict.update({childs.tag:childs.text})
                final_list.append(new_dict)
                
            
                    
                

In [16]:
def create_df_sink_schedule(new_elem,final_list):
#     new_dict = {}
    
    sink_schedule(new_elem,final_list)

In [17]:
def upload_dataframe(final_list,count,data_timestamp):
    doc_df = pd.DataFrame(final_list)
    #             print(doc_df)
    new_columns = []
    for columns in doc_df.columns:
        columns = columns.replace("&","")
        columns = columns.replace(" ","_")
        columns = columns.replace("-","_")
        new_columns.append(columns)
    doc_df.columns = new_columns
    doc_df["upload_date"] = my_timezone.localize(datetime.now()).date() 
    doc_df["data_timestamp"] = pd.to_datetime(data_timestamp[0])
    doc_df["data_timestamp"] = doc_df["data_timestamp"].dt.date
    doc_df.replace(np.nan,None,inplace = True)
    doc_df.replace([np.nan],[None],inplace = True)
    doc_df.replace("nan",None,inplace = True)
    print(len(doc_df))

    #             doc_df.to_pickle(storage_path+"_"+str(count)+".pkl")
    data_loaded_threading(bqclient,doc_df,PROJECT,dataset,table,count)
    
    

In [18]:
import pytz
from datetime import datetime

my_timezone = pytz.timezone('America/New_York')

In [22]:
# count = 0
total = 1086089
columns_list = []
data_timestamp = []
head_context = etree.iterparse(path,events=("start",),tag = "header")
for event,elem in head_context:
    new_elem = etree.parse(BytesIO(etree.tostring(elem)))
    list_of_elem = elem.xpath("//header")
    if len(list_of_elem)>0:
        for element in list_of_elem[0].getchildren():
#             print(element.tag)
            if element.tag == "timestamp":
                print("hello")
                data_timestamp.append(element.text)
    break


def new_fast_iter(context,data_timestamp, *args, **kwargs):
  
    processes = []
    final_list = []
    count = 0
    jobs = []
    for event,elem in context:
        new_elem = etree.parse(BytesIO(etree.tostring(elem)))
        create_df_sink_schedule(new_elem,final_list)
        
        count += 1
    #         elem.clear()
    #         elem.clear()
        while elem.getprevious() is not None:
            del elem.getparent()[0]
#         if count == 10:
#             break
#         print("******")

        if count%10000 == 0:
    #             print(elem.attrib)
            print(count)
        if count %30000== 0:
            upload_dataframe(final_list,count,data_timestamp)
#             thread = threading.Thread(target = threaded_stuff(final_list))

            final_list = []


        if count == total:
            upload_dataframe(final_list,count,data_timestamp)
#             thread = threading.Thread(target = threaded_stuff(final_list))

            final_list = []
            

# def process_element(elem,final_list):
# #     new_elem = elem
    
#     new_elem = etree.parse(BytesIO(etree.tostring(elem)))
# #     print(etree.tostring(elem, pretty_print=True))
# #     print(etree.dump(new_elem))
#     create_df_call_schedule(new_elem,final_list)



context = etree.iterparse(path,events=("start",),tag = "instrument")
new_fast_iter(context,data_timestamp)

hello
10000
20000
30000
12359
success
40000
50000
60000
11773
success
70000
80000
90000
12226
success
100000
110000
120000
12387
success
130000
140000
150000
11696
success
160000
170000
180000
11854
success
190000
200000
210000
12664
success
220000
230000
240000
12048
success
250000
260000
270000
11943
success
280000
290000
300000
11826
success
310000
320000
330000
12091
success
340000
350000
360000
11984
success
370000
380000
390000
12290
success
400000
410000
420000
11063
success
430000
440000
450000
12288
success
460000
470000
480000
12073
success
490000
500000
510000
12188
success
520000
530000
540000
12155
success
550000
560000
570000
11623
success
580000
590000
600000
12092
success
610000
620000
630000
12050
success
640000
650000
660000
12801
success
670000
680000
690000
13067
success
700000
710000
720000
11966
success
730000
740000
750000
12378
success
760000
770000
780000
12272
success
790000
800000
810000
12353
success
820000
830000
840000
11770
success
850000
860000
870000
12